<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250527%5D%20FedAvg-MLP-LSTM%20with%2030%20CommRounds%20%26%205%20clients%20on%20WSN-DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FedAvg-MLP-LSTM with 30 Communications Rounds & 5 clients on WSN-DS**

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, InputLayer
from keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

In [2]:
# ----------------------------
# Preprocessing and Partitioning
# ----------------------------
df = pd.read_csv("dataset_WSN-DS.csv").dropna()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = LabelEncoder().fit_transform(df[col])

X, y = df.drop('Class', axis=1), df['Class']
X = StandardScaler().fit_transform(X)
X, y = SMOTE(random_state=42).fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [3]:
# ----------------------------
# Federated Setup
# ----------------------------
num_clients = 5
rounds = 30
client_X = np.array_split(X_train, num_clients)
client_y = np.array_split(y_train, num_clients)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_test_cat = to_categorical(y_test)
num_classes = y_test_cat.shape[1]
input_shape = (X_test.shape[1], 1)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [4]:
# ----------------------------
# Model Definition
# ----------------------------
def build_model(input_shape, num_classes):
    model = Sequential([
        InputLayer(input_shape=input_shape),
        LSTM(64, activation='tanh'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------------------------
# Training Loop
# ----------------------------
log = []

global_model = build_model(input_shape, num_classes)
global_weights = global_model.get_weights()

for r in range(rounds):
    round_start = time.time()
    client_weights = []
    for i in range(num_clients):
        local_model = build_model(input_shape, num_classes)
        local_model.set_weights(global_weights)
        X_c = client_X[i].reshape(client_X[i].shape[0], client_X[i].shape[1], 1)
        y_c = to_categorical(client_y[i], num_classes=num_classes)
        local_model.fit(X_c, y_c, epochs=1, batch_size=32, verbose=0)
        client_weights.append(local_model.get_weights())

    # FedAvg
    global_weights = [np.mean(np.array([w[i] for w in client_weights]), axis=0) for i in range(len(global_weights))]
    global_model.set_weights(global_weights)

    # Evaluation
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test_cat, axis=1)
    acc = accuracy_score(y_true, y_pred_classes)
    prec = precision_score(y_true, y_pred_classes, average='weighted')
    rec = recall_score(y_true, y_pred_classes, average='weighted')
    f1 = f1_score(y_true, y_pred_classes, average='weighted')
    cm = confusion_matrix(y_true, y_pred_classes)
    auc = roc_auc_score(y_test_cat, y_pred, multi_class='ovr')
    round_time = time.time() - round_start

    log.append({
        'Round': r+1,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'AUC': auc,
        'RoundTime(s)': round_time
    })

    print(f"Round {r+1}: Acc={acc:.8f}, F1={f1:.8f}, AUC={auc:.8f}, Time={round_time:.6f}s")
    print("Confusion Matrix:")
    print(cm)
    print(classification_report(y_true, y_pred_classes))

# ----------------------------
# Save Logs
# ----------------------------
df_log = pd.DataFrame(log)
df_log.to_csv("fedavg_training_log.csv", index=False)
print("Training log saved to fedavg_training_log.csv")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


10628/10628 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step
Round 1: Acc=0.87622697, F1=0.87076527, AUC=0.97530410, Time=433.623537s
Confusion Matrix:
[[66131   947   114   708   113]
 [  440 36700 26963  3910     0]
 [    0     0 67878   135     0]
 [ 4237   342  3675 59760     0]
 [   11   496     0     0 67506]]
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     68013
           1       0.95      0.54      0.69     68013
           2       0.69      1.00      0.81     68013
           3       0.93      0.88      0.90     68014
           4       1.00      0.99      1.00     68013

    accuracy                           0.88    340066
   macro avg       0.90      0.88      0.87    340066
weighted avg       0.90      0.88      0.87    340066



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


10628/10628 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step
Round 2: Acc=0.90620350, F1=0.90295650, AUC=0.98914900, Time=436.384287s
Confusion Matrix:
[[67650   197     2    65    99]
 [   29 41042 26928    14     0]
 [    0     0 67972    41     0]
 [ 4317   162    17 63518     0]
 [   26     0     0     0 67987]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.97     68013
           1       0.99      0.60      0.75     68013
           2       0.72      1.00      0.83     68013
           3       1.00      0.93      0.96     68014
           4       1.00      1.00      1.00     68013

    accuracy                           0.91    340066
   macro avg       0.93      0.91      0.90    340066
weighted avg       0.93      0.91      0.90    340066



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

10628/10628 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step
Round 28: Acc=0.98189175, F1=0.98187819, AUC=0.99902379, Time=441.437779s
Confusion Matrix:
[[67750   113     0    63    87]
 [    6 66465  1542     0     0]
 [    0    69 67944     0     0]
 [ 4261    14     0 63738     1]
 [    2     0     0     0 68011]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     68013
           1       1.00      0.98      0.99     68013
           2       0.98      1.00      0.99     68013
           3       1.00      0.94      0.97     68014
           4       1.00      1.00      1.00     68013

    accuracy                           0.98    340066
   macro avg       0.98      0.98      0.98    340066
weighted avg       0.98      0.98      0.98    340066



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


10628/10628 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step
Round 29: Acc=0.98348556, F1=0.98347233, AUC=0.99908494, Time=444.251395s
Confusion Matrix:
[[67770   102     0    63    78]
 [    8 67055   950     0     0]
 [    0   130 67883     0     0]
 [ 4263    20     0 63731     0]
 [    2     0     0     0 68011]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     68013
           1       1.00      0.99      0.99     68013
           2       0.99      1.00      0.99     68013
           3       1.00      0.94      0.97     68014
           4       1.00      1.00      1.00     68013

    accuracy                           0.98    340066
   macro avg       0.98      0.98      0.98    340066
weighted avg       0.98      0.98      0.98    340066



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


10628/10628 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step
Round 30: Acc=0.98311798, F1=0.98310282, AUC=0.99901451, Time=447.626429s
Confusion Matrix:
[[67766   109     0    56    82]
 [    5 66858  1150     0     0]
 [    0    36 67977     0     0]
 [ 4267    34     0 63713     0]
 [    2     0     0     0 68011]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     68013
           1       1.00      0.98      0.99     68013
           2       0.98      1.00      0.99     68013
           3       1.00      0.94      0.97     68014
           4       1.00      1.00      1.00     68013

    accuracy                           0.98    340066
   macro avg       0.98      0.98      0.98    340066
weighted avg       0.98      0.98      0.98    340066

Training log saved to fedavg_training_log.csv
